In [1]:
import pandas as pd


def load_dataset(path):
    # parse the dataset
    df = pd.read_csv(path)

    ################ distilled from notebook 0 ################
    # check the integrity
    assert df.isna().any().any() == False, 'There is at least one missing value.'
    assert df['Timestamp'].is_monotonic_increasing, 'Timestamp is not sorted.'

    # type-cast
    df['abstime'] = pd.to_datetime(df['Timestamp'], unit='s').round('us')
    df['monotime'] = df['Timestamp'] - df['Timestamp'].min()
    df['aid_int'] = df['Arbitration_ID'].map(lambda x: int(x, 16))
    df['y'] = df['Class'].map({'Normal': 0, 'Attack': 1})

    ################ distilled from notebook 1 ################
    # calculate the stream-wise timedelta
    df['Timedelta'] = df.groupby('Arbitration_ID')['Timestamp'].diff()

    return df


df_stationary = load_dataset('0_Preliminary/0_Training/Pre_train_S_0.csv')
df_driving = load_dataset('0_Preliminary/0_Training/Pre_train_D_0.csv')


#### TASK 7
Write a ode to compute the entropy of the payload.

In [ ]:
import numpy as np
count = df_driving['Arbitration_ID'].value_counts()
p_i = count / df_driving.shape[0]
# compute log of p_i, then you can calculate the entropy.
# you will find numpy useful here


Here, you should write the function get_H.

In [ ]:
get_H(df_stationary.loc[:2000, 'Arbitration_ID']), get_H(df_stationary.loc[10000:20000, 'Arbitration_ID'])
# should get:
# (3.6976371388430116, 3.703190755903826)


In [ ]:
for path in (
        '0_Preliminary/0_Training/Pre_train_S_0.csv',
        '0_Preliminary/0_Training/Pre_train_D_0.csv',
        '0_Preliminary/0_Training/Pre_train_D_1.csv',
):
    print(path, get_H(load_dataset(path)['Arbitration_ID']))

# should get:
# 0_Preliminary/0_Training/Pre_train_S_0.csv 3.703013094908975
# 0_Preliminary/0_Training/Pre_train_D_0.csv 3.7056849451933016
# 0_Preliminary/0_Training/Pre_train_D_1.csv 3.769784213704188


# Sliding window concept


## Calculate $H$ for each window

window size is 2402 (average number of messages per sec), calculate evey 10 messages.
These values are parameters of the IDS. So you may change the value.

In [9]:
df_driving['entropy'] = df_driving.rolling(window=2402, min_periods=2402, step=10)['aid_int'].apply(get_H)
df_driving['entropy'] = df_driving['entropy'].ffill()


In [ ]:
df_driving.plot(x='monotime', y='entropy', figsize=(12, 3))


## Analyze an intrusion dataset

In [11]:
df_intrusion = load_dataset('0_Preliminary/0_Training/Pre_train_D_1.csv')
#Write your code here


In [ ]:
df_intrusion.plot(x='monotime', y='entropy')


# Intrusion detection


In [14]:
df_intrusion['y_predicted'] = 0
rowidx = ~df_intrusion['entropy'].between(3.69, 3.72)
df_intrusion.loc[rowidx, 'y_predicted'] = 1


In [15]:
from sklearn.metrics import confusion_matrix, classification_report

abstime_ceil = df_intrusion['abstime'].dt.ceil('10ms')
y = df_intrusion.groupby(abstime_ceil)['y'].max()
y_predicted = df_intrusion.groupby(abstime_ceil)['y_predicted'].max()

print(confusion_matrix(y, y_predicted))
print(classification_report(y, y_predicted))


[[20003   548]
 [ 3520  6416]]
              precision    recall  f1-score   support

           0       0.85      0.97      0.91     20551
           1       0.92      0.65      0.76      9936

    accuracy                           0.87     30487
   macro avg       0.89      0.81      0.83     30487
weighted avg       0.87      0.87      0.86     30487

